In [1]:
from src import API

API.DataAPI.update()
API.DataAPI.prepare_train_data()
API.Predictor.update_factors()

Update Files
Thu Jun 13 23:51:14 2024 : Updated ~ data\DataBase\DB_information\calendar.feather Done! Cost 0.04 Secs
Thu Jun 13 23:51:14 2024 : Updated ~ data\DataBase\DB_information\description.feather Done! Cost 0.04 Secs
Thu Jun 13 23:51:14 2024 : Updated ~ data\DataBase\DB_information\st.feather Done! Cost 0.01 Secs
Thu Jun 13 23:51:17 2024 : Updated ~ data\DataBase\DB_information\industry.feather Done! Cost 2.49 Secs
Thu Jun 13 23:51:21 2024 : Updated ~ data\DataBase\DB_information\concepts.feather Done! Cost 4.33 Secs
Thu Jun 13 23:51:34 2024 : download since!
Connection and Factor preparation finished!
Thu Jun 13 23:51:34 2024 : sellside/haitong.hf_factors from 20240614 to 20240613, total 1 periods
Start sellside/haitong.hf_factors:20240614-20240613 
Done sellside/haitong.hf_factors:20240614-20240613, cost 1.4 Secs
Thu Jun 13 23:51:35 2024 : sellside/haitong.dl_factors from 20240613 to 20240613, total 1 periods
Start sellside/haitong.dl_factors:20240613-20240613 
Done sellside/h

AssertionError: ['ret10_lag', 'ret20_lag'] not in d:\Coding\learndl\learndl\data\DataBase\DB_labels

In [2]:
from src import API
API.Predictor.update_factors()

Load  2 DataBlocks...... finished! Cost 0.11 secs
Align 2 DataBlocks...... finished! Cost 0.17 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
20231201 new
Load  2 DataBlocks...... finished! Cost 0.03 secs
Align 2 DataBlocks...... finished! Cost 0.15 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
20231201 new
Load  2 DataBlocks...... finished! Cost 0.03 secs
Align 2 DataBlocks...... finished! Cost 0.17 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
20231201 new


In [2]:
from src import API
API.ModelTestor.new('ts_mixer').try_metrics()

Load  2 DataBlocks ...

... cost 0.22 secs
Align 2 DataBlocks ...... cost 0.21 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
x shape is torch.Size([5064, 30, 6])
y shape is torch.Size([5064, 1])
Test Forward Success
metrics :  Metrics.MetricOutput(loss=tensor(1.0688, grad_fn=<AddBackward0>), score=-0.015420470386743546, loss_item=1.068833589553833, penalty=0.0, losses=tensor(1.0688, grad_fn=<ExpBackward0>))
Test Metrics Success
